In [40]:
import pandas as pd
import numpy as np
import json
import ast

***
## Proceso de ETL
***

### **E**xtraction

- La primera parte del proceso consiste en la extraccion de los datos desde su punto de origen, en nuestro caso se encuentran en un conjunto de datasets proporcionados por la empresa steam de tipo json.

In [4]:
games = 'datasets/output_steam_games.json'

# Se lee de cada línea del dataset
rows = []
with open(games) as f:
    for line in f.readlines():
        data = json.loads(line)
        rows.append(data)

# Se convierte a dataframe
games = pd.DataFrame(rows)
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [133]:
reviews = 'datasets/australian_user_reviews.json'

# Se lee de cada línea del dataset
rows.clear()
with open(reviews) as f:
    for line in f.readlines():
        data = ast.literal_eval(line)
        rows.append(data)

# Se convierte a dataframe
reviews = pd.DataFrame(rows)
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [174]:
items = 'datasets/australian_users_items.json'

# Se lee de cada línea del dataset
rows.clear()
with open(items) as f:
    for line in f.readlines():
        data = ast.literal_eval(line)
        rows.append(data)

# Se convierte a dataframe
items = pd.DataFrame(rows)
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


***
### **T**ransform

- La fase de transformacion aplica una serie de reglas de negocio sobre los datos extraidos para poder limpiarlos y tener una buena calidad de estos al momento de cargarlos, la gravedad de estas transformaciones dependera de como se encuentren los datos extraidos. Segun nuestras tablas se requiere de una normalizacion en ellas y de algunos casting para mayor legibilidad y utilidad al momento de realizar la carga y otras operaciones.

#### *Resolver vacios*

In [24]:
print(f'Campos vacios en games: {games.isna().sum().sum()}')
print(f'Campos vacios en reviews: {reviews.isna().sum().sum()}')
print(f'Campos vacios en items: {items.isna().sum().sum()}')

Campos vacios en games: 1168997
Campos vacios en reviews: 0
Campos vacios en items: 0


In [12]:
#Resolvemos
games.dropna(inplace=True)
print(f'Vacios en games: {games.isna().sum().sum()}')

Vacios en games: 0


#### *Normalizacion de columnas*

#### *Games*

In [8]:
games.sample()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
93832,JED GAMES,"[Free to Play, Indie, Massively Multiplayer, R...",Elemental Heroes,Elemental Heroes,http://store.steampowered.com/app/432290/Eleme...,2016-03-03,"[Free to Play, Strategy, RPG, Massively Multip...",http://steamcommunity.com/app/432290/reviews/?...,"[Multi-player, MMO, Cross-Platform Multiplayer...",Free to Play,False,432290,JED GAMES


In [13]:
print(f"Registro iguales entre la columna app_name y title: {(games['app_name']==games['title']).sum()}")
print(f"Cantidad de registros en el dataframe 'games': {len(games)}")

Registro iguales entre la columna app_name y title: 22179
Cantidad de registros en el dataframe 'games': 22530


La mayoria de los valores de una columna se encuentran en la otra, asi que dejaremos una de las dos para que sirva de referencia total al nombre del videojuego y aprovecharemos para eliminar columnas que no utilizaremos

In [16]:
games.drop(columns=['app_name','specs','early_access','publisher','tags','reviews_url'],inplace=True)

El dataframe posee una columna multivaluada que definitivamente nos conviene desglozar para un mejor uso

In [21]:
games = games.explode('genres')

Alguno valores en la columna price no son nisiquiera de tipo flotante, asi que arreglaremos eso cambiando esos valores y casteando la columna

In [31]:
games['price'] = pd.to_numeric(games['price'],errors='coerce').astype(float)
games['price'].fillna(0,inplace=True)

Ultimo retoque para un futuro merge con la tabla item que nos permita unificar las tres tablas

In [208]:
games.rename(columns={"title":"game_title"},inplace=True)

In [36]:
games.sample()

,genres,title,url,release_date,reviews_url,price,id,developer
89651,Indie,Cubemen 2,http://store.steampowered.com/app/228440/Cubem...,2013-04-08,http://steamcommunity.com/app/228440/reviews/?...,7.99,228440,3 Sprockets


#### *Reviews*

In [134]:
reviews.sample()

,user_id,user_url,reviews
11276,76561198165280229,http://steamcommunity.com/profiles/76561198165...,"[{'funny': '', 'posted': 'Posted August 20, 20..."


Otra columna multivaluada

In [135]:
reviews['reviews'].iloc[0]

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [136]:
# Desglosamos
reviews = reviews.explode('reviews')

In [137]:
# Dividimos cada llave de cada diccionario dentro de su propia columna dentro de nuestro dataset
reviews['posted'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['posted'])
reviews['item_id'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['item_id'])
reviews['recommend'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['recommend'])
reviews['review'] = reviews['reviews'].apply(lambda x: np.nan if isinstance(x,float) else x['review'])

In [138]:
print(f"Vacios: {reviews.isna().sum().sum()}")
reviews.dropna(inplace=True)
print(f"Vacios: {reviews.isna().sum().sum()}")

Vacios: 140
Vacios: 0


In [139]:
# Eliminamos la sobrante
reviews.drop(columns=['reviews'],inplace=True)

In [140]:
reviews.sample()

,user_id,user_url,posted,item_id,recommend,review
8927,REBAS_AS_F-T,http://steamcommunity.com/id/REBAS_AS_F-T,"Posted December 28, 2012.",208480,True,trop fort


Las columnas de fechas deberian tener todas un mismo formato

In [142]:
# Dividimos las cadenas de texto en mes dia y año
reviews['posted'] = reviews['posted'].str.replace(',','').str.replace('.','').str.replace('Posted','').str.split()

# Creamos un hashmap para la traduccion de string a numero como referencia del mes 
months = {
  'January': 1,
  'February': 2,
  'March': 3,
  'April': 4,
  'May': 5,
  'June': 6,
  'July': 7,
  'August': 8,
  'September': 9,
  'October': 10,
  'November': 11,
  'December': 12
}
# Array para añadir cada registro ya traducido posteriormente al dataframe
dates=[]
for i,x in enumerate(reviews.itertuples()): # Se itera los registros

  if len(x.posted)==3: # Si el campo posee todo los campos encesarios para tener una fecha entonces se declara la fecha
    date = f"{x.posted[2]}-{months[x.posted[0]]}-{x.posted[1]}" 
  else: 
    date = np.nan

  dates.append(pd.to_datetime(date).date())

reviews['posted'] = pd.DataFrame(dates) # Se añaden a la columna del dataframe

#### *Feature Engineering*

Para una consulta en nuestra API vamos a necesitar una nueva columna que nos especifique cual es el sentimiento que emerge de las palabras de las reviews de los usuarios. Para eso utilizaremos una libreria famosa para el procesamiento de lenguaje natural que nos colaborara en el proceso. 

In [200]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

SID = SentimentIntensityAnalyzer()
analysis = []

for x in reviews.itertuples():
  sentiment = SID.polarity_scores(x.review)['compound']
  if sentiment<-.2:
    value = 0
  elif sentiment>.2:
    value = 2
  else:
    value = 1
  analysis.append(value)

reviews['sentiment'] = pd.DataFrame(analysis)

In [201]:
reviews.sample()

,user_id,user_url,posted,item_id,recommend,review,sentiment
17297,76561198076366524,http://steamcommunity.com/profiles/76561198076...,2014-01-13,200210,True,good fun played on chrome then saw on steam an...,2


#### *Items*

In [175]:
items.sample()

,user_id,items_count,steam_id,user_url,items
35494,jaguarpanda,252,76561198030465641,http://steamcommunity.com/id/jaguarpanda,"[{'item_id': '3920', 'item_name': 'Sid Meier's..."


Otra columna multivariada

In [177]:
# Se divide las listas por registro
items = items.explode('items')

In [178]:
# Dividimos cada llave de cada diccionario dentro de su propia columna dentro de nuestro dataset
items['item_id'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['item_id'])
items['item_name'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['item_name'])
items['playtime_forever'] = items['items'].apply(lambda x: np.nan if isinstance(x,float) else x['playtime_forever'])

In [179]:
# Validamos vacios
print(f"Vacios: {items.isna().sum().sum()}")
items.dropna(inplace=True)
print(f"Vacios: {items.isna().sum().sum()}")

Vacios: 67224
Vacios: 0


In [183]:
# Eliminamos columnas innecesarias
items.drop(columns=['items','steam_id','items_count','user_url'],inplace=True)

In [207]:
# Retoque final para el merge con la tabla games
items.rename(columns={"item_name":"game_title"},inplace=True)

In [206]:
items.sample()

,user_id,item_id,item_name,playtime_forever
53821,dwyn190,55230,Saints Row: The Third,0.0


***
### **L**oad
- Como cereza del pastel debemos cargar nuestros datos ya procesados a un sistema de origen. Este sistema dependera bastante de los requisitos de la organizacion y varia segun sus objetivos. Para este proyecto la carga se hara directamente en la Web mediante una API y en esta ocasion exportaremos solo parte de nuestro gran dataset, por cuestion de rendimiento, eficiencia y funcionamiento de la aplicacion que utilizaremos para el despliegue, Render. Para ser mas claros estos servicios poseen una prueba gratuita pero bastante limitada en cuestion de memoria.

In [210]:
games.sample()

,genres,game_title,url,release_date,reviews_url,price,id,developer
114025,Sports,MiniDrivers,http://store.steampowered.com/app/385490/MiniD...,2015-08-20,http://steamcommunity.com/app/385490/reviews/?...,5.99,385490,Ivanovich Games


In [209]:
items.sample()

,user_id,item_id,game_title,playtime_forever
52902,Cropmy5,47780,Dead Space 2,0.0


In [203]:
reviews.sample()

,user_id,user_url,posted,item_id,recommend,review,sentiment
18732,KingOfArrows,http://steamcommunity.com/id/KingOfArrows,2015-01-16,210970,True,* I have not once ever felt bored with this ga...,2
